# notes

#re-add the averaging for test values due to each fold having different testing data
#add multi-layer perceptron for secondary classifier
#add functionality for "leave one out cross validation", more effective with small sample sizes.
above is complete



# predict label

## step 1

In [10]:


import os
os.chdir()
#change to your directory
import pandas as pd
!pip install torch torchvision torchaudio
!pip install lightGBM
!pip install statsmodels
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor,AdaBoostClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, jaccard_score, matthews_corrcoef, confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
kf = KFold(n_splits=96, shuffle=True, random_state=23)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix
import json


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## step 2

In [11]:
def workflow(x, USEDmodels, USEDmodelnames, randomstate):
    x = x.sample(frac=1, random_state=randomstate).reset_index(drop=True)

    y = x['subtype']

    
    
   
    
    #CVgridsearch can go here when needed
    
    
    


    #print(y)
    ytrain = y
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    #print(y)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    x = x.drop(['subtype'], axis=1)
    xtrain = x
    print(f"Random state: {randomstate}")
    #print(x, y)
    def resetidx(x):
        x.reset_index(drop=True, inplace=True)  # Reset the index
        return x

    resetidx(xtrain)
    resetidx(ytrain)
    #print(xtrain, ytrain)

    #Block A
    import time
    start_time = time.time()
    stacked = "individual models"
    models = USEDmodels
    model_names = USEDmodelnames
    print(model_names)

    oof_prob_pred, oofTrueLabels, oof_pred, stacked, oofOGdata = get_oof(xtrain, ytrain, stacked, models, model_names, randomstate)

    end_time = time.time()
    
    total_time = end_time - start_time
    print(f"运行时间为: {total_time:.2f} 秒")

    #Block B
    #get oof probabilities into the right order.
    #these were combined through adding each set of #subtypenumber as a new column for the #classifiers created datasets for a total of subtype x classifiers.
    models = OGstack_models
    model_names = OGstack_model_names
    oof_prob_pred_combined1 = np.concatenate([oof_prob_pred[key] for key in oof_prob_pred], axis=1)
    print("combining", oof_prob_pred_combined1, oofOGdata)
    oof_prob_pred_combined1 = np.concatenate([oof_prob_pred_combined1, oofOGdata], axis=1) #CURRENT STATUS:COMBINING
    oof_prob_pred_combined1 = pd.DataFrame(oof_prob_pred_combined1)

    stacked = "stacked"

    #makes it into a list for the initial code to easier read it
    oof_prob_pred2, oofTrueLabels2, oof_pred2, stacked, oofOGdata = get_oof(oof_prob_pred_combined1, oofTrueLabels, stacked, models, model_names, randomstate)


## step 3

In [12]:

def evaluate_classifier(Truelabels, PredictedLabels, reg, ProbaPredictions, stacked, model_name, randomstate):
    
    reverse_mapping = {v: k for k, v in mapping.items()}
    Tl = Truelabels.map(reverse_mapping)
    Tl.name = 'subtype'
    ProP = pd.DataFrame(ProbaPredictions)
    ProP = ProP.rename(columns=reverse_mapping)
    predictions = pd.concat([Tl, ProP], axis=1)
    
    accuracy = accuracy_score(Truelabels, PredictedLabels)
    precision = precision_score(Truelabels, PredictedLabels, average='weighted')
    recall = recall_score(Truelabels, PredictedLabels, average='weighted')
    f1 = f1_score(Truelabels, PredictedLabels, average='weighted')
    mcc = matthews_corrcoef(Truelabels, PredictedLabels)
    if len(Truelabels) > (len(x)/2):
        print("")
        print("model score of oof results for", model_name)
        auc = roc_auc_score(Truelabels, ProbaPredictions, multi_class='ovr', average='weighted')
        testing = "val_data"
        #print("true labels", Truelabels,"predicted labels", PredictedLabels)
    cm = confusion_matrix(Truelabels, PredictedLabels)
    if cm.shape == (2, 2):  # Binary classification
        tn, fp, fn, tp = cm.ravel()

        # Sensitivity and Specificity for binary classification
        Sensitivity = tp / (tp + fn)
        Specificity = tn / (tn + fp)

    else:  # Multi-class classification
        sensitivities = []
        specificities = []

        # Calculate for each class
        for i in range(cm.shape[0]):  # Loop over each class
            tp = cm[i, i]
            fn = cm[i].sum() - tp  # All positives for this class
            fp = cm[:, i].sum() - tp  # All predicted positives for this class
            tn = cm.sum() - (fp + fn + tp)  # Total - (FP + FN + TP)

            # Sensitivity for this class
            sensitivity1 = tp / (tp + fn) if (tp + fn) > 0 else 0
            sensitivities.append(sensitivity1)

            # Specificity for this class
            specificity1 = tn / (tn + fp) if (tn + fp) > 0 else 0
            specificities.append(specificity1)

        # Average sensitivity and specificity across classes
        sensitivity = np.mean(sensitivities)
        specificity = np.mean(specificities)

    
    if cm.shape == (2, 2):  # Binary classification
        tn, fp, fn, tp = cm.ravel()

        # G-measure for binary classification
        G_measure = np.sqrt((tp / (tp + fn)) * (tn / (tn + fp)))

        # Jaccard for binary classification
        Jaccard = tp / (tp + fp + fn)

    else:  # Multi-class classification
        G_measures = []
        Jaccard_testices = []

        # Calculate for each class
        for i in range(cm.shape[0]):  # Loop over each class
            tp = cm[i, i]
            fn = cm[i].sum() - tp  # All positives for this class
            fp = cm[:, i].sum() - tp  # All predicted positives for this class
            tn = cm.sum() - (fp + fn + tp)  # Total - (FP + FN + TP)

            # G-measure for this class
            G_measure = np.sqrt((tp / (tp + fn)) * (tn / (tn + fp))) if (tp + fn) > 0 and (tn + fp) > 0 else 0
            G_measures.append(G_measure)

            # Jaccard for this class
            Jaccard = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
            Jaccard_testices.append(Jaccard)

        # Average G-measure and Jaccard index across classes
        g_measure = np.mean(G_measures)
        jaccard = np.mean(Jaccard_testices)
    # Now G_measure and Jaccard contain the values for your model

# Print all metrics
    print("Model Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall (Sensitivity):", recall)
    print("F1 Score:", f1)
    print("Matthews Correlation Coefficient (MCC):", mcc)
    print("G-Measure:", g_measure)
    print("AUC:", auc)
    print("Specificity:", specificity)
    print("Jaccard Index:", jaccard)


    metrics = {
    "Accuracy": [accuracy],
    "Precision": [precision],
    "Recall (Sensitivity)": [sensitivity],
    "F1 Score": [f1],
    "MCC": [mcc],
    "G-Measure": [g_measure],
    "AUC": [auc],
    "Specificity": [specificity],
    "Jaccard Index": [jaccard],
    }

# Convert metrics to DataFrame and save as CSV
    metrics_df = pd.DataFrame(metrics).T
    
    
    
    # Define base file path and subdirectories
    
    #filepath = '//' #for combination testing
    folderpath = "//"
    #folderpath = "/" #for combination testing
    foldername = f"{run}/"
    filepath = f"{folderpath}{foldername}"
    subdir = f"{testing}/{stacked}/{model_name}/"    
    filename = f"{data}({randomstate}),metrics.csv"
    filename2 = f"predictions.csv"
    full_path = os.path.join(filepath, subdir, filename)  #evaluation
    full_path2 = os.path.join(filepath, subdir, filename2) #predictions
    
    # Check if the subdirectory exists; if not, create it
    if not os.path.exists(os.path.dirname(full_path)):
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        print(f"Created directory: {os.path.dirname(full_path)}")
    
    print(f"Saving metrics to: {full_path}")
    print("PREDICTIONS", predictions)
    # Save the metrics DataFrame to CSV
    metrics_df.to_csv(full_path, index=False)
    predictions.to_csv(full_path2) 
    return metrics


def oofindividualscore(X, models, oofTrueLabels, oof_pred, stacked, model_names, randomstate):
    #it is in dictionary format and will automatically reference set of predictions for a particular model by name.
    for idx, model in enumerate(models):
        model_name = model_names[idx]
        classy = oof_pred[model_name]
        x1 = X[model_name]
        #print("true labels", oofTrueLabels, "predicted labels", classy, model, "probabilities", x1, stacked, model_name)
        evaluate_classifier(oofTrueLabels, classy, model, x1, stacked, model_name, randomstate)
        

def get_oof(xtrain, ytrain, stacked, models, model_names, randomstate):
    #note: empty arrays initialized so they can take shape as data is added to them. 
    #allows for more flexibility when removing or adding models
    oofTrueLabels = pd.Series(dtype=int)
    oofOGdata = pd.DataFrame()
    oof_pred = {}
    oof_prob_pred = {}

    #splits training data into training and validation indexs
    for fold_idx, (train_index, val_index) in enumerate(kf.split(xtrain, ytrain)): 
        kf_x_train = xtrain.iloc[train_index]
        kf_y_train = ytrain[train_index]
        kf_x_val = xtrain.iloc[val_index]
        kf_y_val = ytrain[val_index]
        oof_predictions = pd.Series(index=val_index, dtype=int)
        # Ensure 'data' is used as a string key
        print(stacked)
        if stacked == "individual models":
            oofOGdataNew = kf_x_val
            oofOGdata = pd.concat([oofOGdata, oofOGdataNew], axis=0)

        # Check if the key exists in the dictionary, then update accordingly
        # Check if validation data already exists in the dictionary
    
        #print("val_index", val_index)
        #print("test index", test_index)

        for idx, model in enumerate(models):

            model.fit(kf_x_train, kf_y_train) #training the model
            model_name = model_names[idx]
            proba_val = model.predict_proba(kf_x_val)
            oof_predictions[val_index] = np.argmax(proba_val, axis=1)  # Assuming max probability class 
            #this section builds the arrays and dictionaries, adding and shaping it as it goes.
            #if an array does not exist for a model, it will be created.
            #if it does, values/probabilities are added as rows.
            if model_name in oof_pred:
                #print("Adding new labels to the existing true label Series")
                oof_pred[model_name] = pd.concat([oof_pred[model_name], pd.Series(oof_predictions[val_index])], ignore_index=True)
            else:
                #print("Creating a new true label Series for this model")
                oof_pred[model_name] = pd.Series(oof_predictions[val_index])
            if model_name in oof_prob_pred:
                #print("Adding the new oof predictions with the existing list for the model")
                oof_prob_pred[model_name] = np.concatenate([oof_prob_pred[model_name], proba_val], axis=0)
                #print(oof_prob_pred)
            else:
                #print("Creating a new oof data entry in the dictionary for the model")
                oof_prob_pred[model_name] = proba_val  # Directly assign
                #print(oof_prob_pred)
    
                
        print("cv")
        # assembles a list of the corresponding true labels for the oof data
        oofTrueLabelsCurrent = kf_y_val  
        oofTrueLabels = pd.concat([oofTrueLabels, oofTrueLabelsCurrent], ignore_index=True)
    
    #print(oof_prob_pred, oofTrueLabels, oof_pred, test_prob_pred_mean, test_pred, stacked)
    num_models = len(models)
    
    
    oofOGdata_numpy = oofOGdata.to_numpy()
    
    #evaluate the performance of the individual classifiers
    oofindividualscore(oof_prob_pred, models, oofTrueLabels, oof_pred, stacked, model_names, randomstate)
    
    #evaluate the test score for each model
    return oof_prob_pred, oofTrueLabels, oof_pred, stacked, oofOGdata_numpy



        
    
        

## step 4

In [ ]:
#models = [SVC(kernel="rbf", probability=True), XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'), RandomForestClassifier(random_state=0, n_estimators=100)]
#OGmodels = [SVC(kernel="rbf", probability=True), XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'),RandomForestClassifier(random_state=0, n_estimators=100)]
#OGmodels = [XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'), GaussianNB()]

mlpparameters = {
"activation": "relu",
"alpha": 0.0001,
"learning_rate": "constant",
"max_iter": 210,
"batch_size": 8,
"solver": "adam",

"hidden_layer_sizes": (100, 100)
}

#QDA('reg_param'= 1.0, 'store_covariance'= True, 'tol'= 0.000)
OGmodels = [SVC(kernel="rbf", probability=True),
            SVC(kernel="linear", probability=True)
            #LogisticRegression(max_iter=700)
            #GaussianNB(),
            #RandomForestClassifier(random_state=0, n_estimators=100),
            #XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'),
            #MLPClassifier(**mlpparameters),
            #KNeighborsClassifier(),
            #QuadraticDiscriminantAnalysis(reg_param= 1.0, store_covariance= True, tol= 0.000),
            #DecisionTreeClassifier()
           ]
           
    
OGstack_models = [
    #MLPClassifier(**mlpparameters),
    #RandomForestClassifier(),
    #LogisticRegression(max_iter=700),
    #GaussianNB(),
    SVC(kernel="rbf", probability=True),
    SVC(kernel="linear", probability=True)
    #MLPClassifier(activation='relu', alpha= 0.0001, batch_size= 1, hidden_layer_sizes=(200,), learning_rate= 'constant', solver= 'adam', max_iter=400)
]
number_models = len(OGmodels)
model_names = [type(model).__name__ for model in OGmodels]
# Dictionary to track occurrences of each model name
name_count = {}
# Updated list to hold the unique model names
unique_model_names = []

for name in model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    # Append the unique name to the result list
    unique_model_names.append(unique_name)
OGmodel_names=unique_model_names


stack_model_names = [type(model).__name__ for model in OGstack_models]
# Updated list to hold the unique model names

unique_stack_model_names = []
for name in stack_model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    unique_stack_model_names.append(unique_name)
OGstack_model_names = unique_stack_model_names

print(OGmodel_names)
print(OGstack_model_names)
USEDmodels = OGmodels
USEDmodelnames = OGmodel_names
    
    

    
le = LabelEncoder()
x = pd.read_csv('') #your file here
data = "" #give the data a name
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

 #Define the mapping
mapping = {
    'Squamous': 0,
    'ADEX': 1,
    'Pancreatic Progenitor': 2,
    'Immunogenic': 3
}

# Map the values
x['subtype'] = x['subtype'].map(mapping)

classnumber = len(np.unique(x['subtype']))
#print(classnumber)
randomstate = 0
# List of states
states = list(range(20))
run = 0
print("run", run)
# Loop through each state and run the workflow
for state in states:
    workflow(x, USEDmodels, USEDmodelnames, randomstate=state)
    run = run+1
    

['SVC', 'SVC2']
['SVC3', 'SVC4']
run 0
Random state: 0
['SVC', 'SVC2']
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
individual models
cv
indiv

## step 4 (2 model combinations)

In [9]:
# run for all combinations of 2 models
# Define your models
import itertools


run = 0


mlpparameters = {
"activation": "relu",
"alpha": 0.0001,
"learning_rate": "constant",
"max_iter": 210,
"batch_size": 8,
"solver": "adam",

"hidden_layer_sizes": (100, 100)
}
OGmodels = [
    SVC(kernel="rbf", probability=True),
    SVC(kernel="linear", probability=True),
    LogisticRegression(max_iter=700),
    GaussianNB(),
    XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'),
    KNeighborsClassifier(),
    QuadraticDiscriminantAnalysis(reg_param=1.0, store_covariance=True, tol=0.000)
]


OGstack_models = [
    #MLPClassifier(**mlpparameters),
    #RandomForestClassifier(),
    #LogisticRegression(max_iter=700),
    #GaussianNB(),
    SVC(kernel="rbf", probability=True),
    SVC(kernel="linear", probability=True),
    #MLPClassifier(activation='relu', alpha= 0.0001, batch_size= 1, hidden_layer_sizes=(200,), learning_rate= 'constant', solver= 'adam', max_iter=400)
]

number_models = len(OGmodels)
model_names = [type(model).__name__ for model in OGmodels]
# Dictionary to track occurrences of each model name
name_count = {}
# Updated list to hold the unique model names
unique_model_names = []

for name in model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    # Append the unique name to the result list
    unique_model_names.append(unique_name)
OGmodel_names=unique_model_names


stack_model_names = [type(model).__name__ for model in OGstack_models]
# Updated list to hold the unique model names

unique_stack_model_names = []
for name in stack_model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    unique_stack_model_names.append(unique_name)
OGstack_model_names = unique_stack_model_names

print(OGmodel_names)
print(OGstack_model_names)



# Create a dictionary that maps model names to model instances
OGmodel_dict = dict(zip(OGmodel_names, OGmodels))

# Print the combined dictionary
print("dict", OGmodel_dict)

OGstack_model_dict = dict(zip(OGstack_model_names, OGstack_models))

# Print the combined dictionary
print("dict", OGstack_model_dict)




# Create all combinations of 2 classifiers from OGmodels
model_combinations = list(itertools.combinations(OGmodels, 2))
#print("model combinations", model_combinations)

# Iterate over each combination of models
for model1, model2 in model_combinations:
    print(f"Model Pair: {type(model1).__name__} + {type(model2).__name__}")
    USEDmodels = [model1, model2]
    
    # Function to find the key corresponding to a model
    def find_model_key(model_dict, input_model):
        # Get the input model's parameters
        input_params = input_model.get_params()

        # Iterate through the dictionary
        for key, model in model_dict.items():
            # Compare parameters of the current model in the dictionary with the input model
            if model.get_params() == input_params:
                return key  # Return the key if the model parameters match

        return None  # Return None if no match is found

    # Find the key for the input model
    

    
    model_key1 = find_model_key(OGmodel_dict, model1)
    model_key2 = find_model_key(OGmodel_dict, model2)
    
    
        #add the svms here
    model3 = SVC(kernel="rbf", probability=True),
    model4 = SVC(kernel="linear", probability=True)
    
    model_key3 = type(model3).__name__
    model_key4 = type(model4).__name__
    
    
  
    USEDmodelnames = [model_key1, model_key2, model_key3, model_key4]
    print("USEDmodelnames", USEDmodelnames)
    
    le = LabelEncoder()
    x = pd.read_csv('') #your file here
    data = "" #give the data a name
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    #print(x['subtype'])
    #x['subtype'] = le.fit_transform(x['subtype'])

     #Define the mapping
    mapping = {
        'Squamous': 0,
        'ADEX': 1,
        'Pancreatic Progenitor': 2,
        'Immunogenic': 3
    }

    # Map the values
    x['subtype'] = x['subtype'].map(mapping)

    classnumber = len(np.unique(x['subtype']))
    #print(classnumber)
    randomstate = 0
    # List of states
    states = list(range(1))
    print(f"Running with: {type(model1).__name__} and {type(model2).__name__}")
    models = {model1, model2}
    # Loop through each state and run the workflow
    for state in states:
        workflow(x, USEDmodels, USEDmodelnames, randomstate=state)
    
    # Placeholder: print model names for demonstration
    run = run+1
    print("run", run)
    
    # Process the models or create a stack of models
    # Example of stacking:
    # stack_model = [model1, model2]  # or any other combination logic
    # You can add your desired workflow for each model pair here
    
    
    

['SVC', 'SVC2', 'LogisticRegression', 'GaussianNB', 'XGBClassifier', 'KNeighborsClassifier', 'QuadraticDiscriminantAnalysis']
['SVC3', 'SVC4']
dict {'SVC': SVC(probability=True), 'SVC2': SVC(kernel='linear', probability=True), 'LogisticRegression': LogisticRegression(max_iter=700), 'GaussianNB': GaussianNB(), 'XGBClassifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_e

KeyboardInterrupt: 

## step 4 (4 model combinations)

In [ ]:
# run for all combinations of 4 models
# Define your models
import itertools


run = 0


mlpparameters = {
"activation": "relu",
"alpha": 0.0001,
"learning_rate": "constant",
"max_iter": 210,
"batch_size": 8,
"solver": "adam",

"hidden_layer_sizes": (100, 100)
}
OGmodels = [
    SVC(kernel="rbf", probability=True),
    SVC(kernel="linear", probability=True),
    LogisticRegression(max_iter=700),
    GaussianNB(),
    XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss'),
    KNeighborsClassifier(),
    QuadraticDiscriminantAnalysis(reg_param=1.0, store_covariance=True, tol=0.000)
]


OGstack_models = [
    MLPClassifier(**mlpparameters),
    RandomForestClassifier(),
    LogisticRegression(max_iter=700),
    GaussianNB(),
    SVC(kernel="rbf", probability=True),
    SVC(kernel="linear", probability=True),
    #MLPClassifier(activation='relu', alpha= 0.0001, batch_size= 1, hidden_layer_sizes=(200,), learning_rate= 'constant', solver= 'adam', max_iter=400)
]

number_models = len(OGmodels)
model_names = [type(model).__name__ for model in OGmodels]
# Dictionary to track occurrences of each model name
name_count = {}
# Updated list to hold the unique model names
unique_model_names = []

for name in model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    # Append the unique name to the result list
    unique_model_names.append(unique_name)
OGmodel_names=unique_model_names


stack_model_names = [type(model).__name__ for model in OGstack_models]
# Updated list to hold the unique model names

unique_stack_model_names = []
for name in stack_model_names:
    if name in name_count:
        # If the name is already seen, increment the count and append the number
        name_count[name] += 1
        unique_name = f"{name}{name_count[name]}"
    else:
        # If the name is seen for the first time, just use the name
        name_count[name] = 1
        unique_name = name
    unique_stack_model_names.append(unique_name)
OGstack_model_names = unique_stack_model_names

print(OGmodel_names)
print(OGstack_model_names)



# Create a dictionary that maps model names to model instances
OGmodel_dict = dict(zip(OGmodel_names, OGmodels))

# Print the combined dictionary
print("dict", OGmodel_dict)

OGstack_model_dict = dict(zip(OGstack_model_names, OGstack_models))

# Print the combined dictionary
print("dict", OGstack_model_dict)




# Create all combinations of 2 classifiers from OGmodels
model_combinations = list(itertools.combinations(OGmodels, 4))
#print("model combinations", model_combinations)

# Iterate over each combination of models
for model1, model2, model3, model4 in model_combinations:
    print(f"Model Pair: {type(model1).__name__} + {type(model2).__name__} + {type(model3).__name__} + {type(model4).__name__}")
    USEDmodels = [model1, model2, model3, model4]
    
    # Function to find the key corresponding to a model
    def find_model_key(model_dict, input_model):
        # Get the input model's parameters
        input_params = input_model.get_params()

        # Iterate through the dictionary
        for key, model in model_dict.items():
            # Compare parameters of the current model in the dictionary with the input model
            if model.get_params() == input_params:
                return key  # Return the key if the model parameters match

        return None  # Return None if no match is found

    # Find the key for the input model
    model_key1 = find_model_key(OGmodel_dict, model1)
    model_key2 = find_model_key(OGmodel_dict, model2)
    model_key3 = find_model_key(OGmodel_dict, model3)
    model_key4 = find_model_key(OGmodel_dict, model4)
    
    
    USEDmodelnames = [model_key1, model_key2, model_key3, model_key4]
    print("USEDmodelnames", USEDmodelnames)
    
    le = LabelEncoder()
    x = pd.read_csv('') #your file here
    data = "" #give the data a name
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
   
    #Define the mapping
    mapping = {
        'Squamous': 0,
        'ADEX': 1,
        'Pancreatic Progenitor': 2,
        'Immunogenic': 3
    }

    # Map the values
    x['subtype'] = x['subtype'].map(mapping)

    classnumber = len(np.unique(x['subtype']))
    #print(classnumber)
    randomstate = 0
    # List of states
    states = list(range(1))
    print(f"Running with: {type(model1).__name__} and {type(model2).__name__} and {type(model3).__name__} and {type(model4).__name__}")
    models = {model1, model2, model3, model4}
    # Loop through each state and run the workflow
    for state in states:
        workflow(x, USEDmodels, USEDmodelnames, randomstate=state)
    
    # Placeholder: print model names for demonstration
    run = run+1
    print("run", run)
    
    # Process the models or create a stack of models
    # Example of stacking:
    # stack_model = [model1, model2]  # or any other combination logic
    # You can add your desired workflow for each model pair here
    
    
    

['SVC', 'SVC2', 'LogisticRegression', 'GaussianNB', 'XGBClassifier', 'KNeighborsClassifier', 'QuadraticDiscriminantAnalysis']
['MLPClassifier', 'RandomForestClassifier', 'LogisticRegression2', 'GaussianNB2', 'SVC3', 'SVC4']
dict {'SVC': SVC(probability=True), 'SVC2': SVC(kernel='linear', probability=True), 'LogisticRegression': LogisticRegression(max_iter=700), 'GaussianNB': GaussianNB(), 'XGBClassifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=Non

/home/wanlab/nick/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [09:21:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


cv
individual models


/home/wanlab/nick/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [09:21:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


cv
individual models


...

/home/wanlab/nick/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:09:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


cv
individual models


/home/wanlab/nick/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:09:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


cv
individual models


/home/wanlab/nick/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [13:10:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## CVgridsearch

In [ ]:
# CVgridsearch

# Define the MLPClassifier
    mlp = MLPClassifier(max_iter=1000, random_state=42)

    # Define the parameter grid for GridSearchCV
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (150,), (200,)],  # Number of neurons in the hidden layer(s)
        'activation': ['relu', 'tanh', 'logistic'],              # Activation functions
        'solver': ['adam', 'sgd'],                               # Optimization algorithm
        'learning_rate': ['constant', 'invscaling', 'adaptive'],  # Learning rate schedule
        'alpha': [0.0001, 0.001, 0.01],                          # Regularization term
        'batch_size': [32, 64, 128]                              # Batch size
    }

        param_grid = {
        'hidden_layer_sizes': [(50,50), (100,100), (150,150)],  # Different sizes of hidden layers
        'activation': ['relu', 'tanh'],  # Activation functions
        'solver': ['adam', 'sgd'],  # Solvers for optimization
        'alpha': [0.0001],  # Regularization parameter
        'learning_rate': ['constant', 'invscaling', 'adaptive'],  # Learning rate schedule
        'batch_size': [1, 16, 32, 64, 96],  # Different batch sizes to test
        'max_iter': [200, 300]  # Number of iterations for training
    }
    
    # Initialize the GridSearchCV
    grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

    # Fit the GridSearchCV to find the best parameters
    grid_search.fit(oof_prob_pred_combined1, oofTrueLabels2)

    # Print the best parameters found
    print("Best parameters found: ",stacked, grid_search.best_params_)

    # Evaluate the best model on the test set
    best_mlp = grid_search.best_estimator_
    y_pred = best_mlp.predict(oof_prob_pred_combined1)
    # Print classification report for evaluation
    print(classification_report(oofTrueLabels2, y_pred))

In [ ]:
# CVgridsearch

    X = x # Features

    # Split your dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Define the parameter grid for QuadraticDiscriminantAnalysis
    param_grid = {  # Class prior probabilities
        'reg_param': [0.0, 0.1, 0.5, 0.7, 1.0],  # Regularization parameter for covariance regularization
        'store_covariance': [True, False],  # Whether to store the covariance matrix
        'tol': [1e-4, 1e-5, 1e-6],  # Tolerance for covariance matrix inversion
    }


    # Initialize the QuadraticDiscriminantAnalysis model
    qda = QuadraticDiscriminantAnalysis()

    # Set up the GridSearchCV with 5-fold cross-validation
    grid_search = GridSearchCV(estimator=qda, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

    # Fit the grid search to the training data
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_qda = grid_search.best_estimator_

    # Print the best parameters found
    print("Best parameters found:", grid_search.best_params_)

    # Get the best parameters from GridSearchCV
    best_params = grid_search.best_params_

    # Define the file path where you want to save the parameters
    file_path = '/work/wanlab/nick/grid/best_params_qda.json'

    # Write the dictionary to a JSON file
    with open(file_path, 'w') as f:
        json.dump(best_params, f, indent=4)

    # Verify the saved content
    print(f"Best parameters saved to {file_path}")

    # Evaluate the model on the test set
    y_pred = best_qda.predict(X_test)
    print("Classification report on the test set:")
    print(classification_report(y_test, y_pred))

    # Optionally, you can also print the best score from the cross-validation
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    

In [ ]:
# CVgridsearch

    X = x # Features
    # Split your dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    # Define the parameter grid for the MLPClassifier
    mlpparameters = {
    "activation": "relu",
    "alpha": 0.0001,
    "learning_rate": "constant",
    "max_iter": 210,
    "batch_size": 8,
    "solver": "adam",
    "random_state": 42,
    "hidden_layer_sizes": (100, 100)
    }
    param_grid = {
    "hidden_layer_sizes": [(50,50), (100, 100), (200, 200), (400, 400)]
    }

    # Initialize the MLPClassifier
    mlp = MLPClassifier(**mlpparameters)

    # Set up the GridSearchCV with 5-fold cross-validation
    grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=13, n_jobs=-1, verbose=2)

    # Fit the grid search to the training data
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_mlp = grid_search.best_estimator_

    # Print the best parameters found
    print("Best parameters found:", grid_search.best_params_)

    # Get the best parameters from GridSearchCV
    best_params = grid_search.best_params_

    # Define the file path where you want to save the parameters
    file_path = '/work/wanlab/nick/grid/best_params_mlp.json'

    # Write the dictionary to a JSON file
    with open(file_path, 'w') as f:
        json.dump(best_params, f, indent=4)

    # Verify the saved content
    print(f"Best parameters saved to {file_path}")
    
    # Evaluate the model on the test set
    y_pred = best_mlp.predict(X_test)
    print("Classification report on the test set:")
    print(classification_report(y_test, y_pred))

    # Optionally, you can also print the best score from the cross-validation
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")